# Modularbeit Künstliche Intelligenz
## Name: Arrif Sondjalim
## Studiengang: DE 
## Matrikelnummer: 26959622

In [1]:
import random
import time
from IPython.display import clear_output
# import heapq
from collections import deque

In [2]:
'''
Definition von Objekten und Feldtypen
'''
WALL = '#'
SPACE = ' '
NEST = 'N'
FOOD = 'o'
ANT = 'A'
ANT_WITH_FOOD = 'C'

In [3]:
'''
Ameisenklasse
'''
class Ant:
    def __init__(self, environment, x, y):
        self.x = x # X-Position in der Environment
        self.y = y # Y-Position in der Environment
        self.environment = environment
        self.carry = '' # Was trägt die Ameise gerade, '' für nichts
        self.directions = [(0, 1), (1, 0), (0, -1), (-1, 0)] # Zufällige Bewegungsrichtung: Oben, Rechts, Unten, Links
        
    # eine Ameise kann sich ein Feld nach oben, unten, links oder rechts bewegen    
    def walk(self, dx, dy):
        if abs(dx) + abs(dy) > 1:
            return
        if self.environment.getFloor(self.x + dx, self.y + dy) != WALL:
            self.x += dx
            self.y += dy
    
    # eine Ameise kann wahrnehmen, auf welchem Feld sie sich befindet
    # - einem leeren Feld: SPACE
    # - dem Nest NEST
    # - einem Feld mit Essen FOOD
    def sense(self):
        return self.environment.getFloor(self.x, self.y)
    
    # trägt die Ameise gerade Essen mit sich?
    def carryingFood(self):
        return self.carry == FOOD
    
    # nehme Essen auf
    def take(self):
        if self.carry == '' and self.sense() == FOOD:
            self.carry = FOOD
            self.environment.setFloor(self.x, self.y, SPACE)
      
    # lasse das Essen fallen
    def drop(self):
        if self.carry != '':
            if self.sense() == NEST:
                self.environment.setFloor(self.x, self.y, SPACE)
            else:
                self.environment.setFloor(self.x, self.y, self.carry)
            self.carry = ''
    
    def random_walk(self):
        """
        Beschreibung: Fuehrt eine Bewegung der Ameise in eine zufaellige Himmelsrichtung durch
        Returns: None
        """
        dx, dy = random.choice(self.directions)
        self.walk(dx, dy)

    def find_target(self, target_type):
        """
        Beschreibung: Sucht das nächstgelegene Ziel des gegebenen Typs mithilfe der Bestensuche
        Parameter: `target_type` als Zielort
        Returns: Ort des nächstgelegenen Ziels vom Typ `target_type` oder `None`, wenn das Ziel nicht gefunden werden kann
        """
        queue = deque()
        start = (self.x, self.y)
        queue.append((start, None))  # Speichern des Vorgängers für jeden Knoten
        visited = set()
        breadcrumbs = {}  # Um auf Nummer sicher zu gehen: Rückverfolgung
        
        while queue:
            current, predecessor = queue.popleft()
            if current in visited:
                continue
            visited.add(current)
            breadcrumbs[current] = predecessor

            if self.environment.getFloor(*current) == target_type:
                path = []
                while current:
                    path.append(current)
                    current = breadcrumbs[current]
                return path[::-1]  # Umgekehrter Pfad zum Ziel

            neighbors = [(current[0] + dx, current[1] + dy) for dx, dy in self.directions]
            for next_cell in neighbors:
                if self.environment.getFloor(*next_cell) != WALL and next_cell not in visited:
                    queue.append((next_cell, current))
                    
        return None

    def step_towards(self, path):
        """
        Beschreibung: Bewegung der Ameise entlang eines Pfades.
        Parameter: `path`: Der Pfad, den die Ameise entlanglaufen soll.
        Returns: None
        """
        if path and len(path) > 1:  # Stelle sicher, dass ein Pfad existiert und mehr als ein Schritt notwendig ist
            next_step = path[1]  # Der nächste Schritt auf dem Pfad
            self.walk(next_step[0] - self.x, next_step[1] - self.y)

    def act(self):
        """Aktion dermeise, abhängig von ihrem Zustand und ihrer Umgebung."""
        if self.carryingFood():
            if self.sense() == NEST:
                self.drop()
            else:
                path_to_nest = self.find_target(NEST)
                if path_to_nest:
                    self.step_towards(path_to_nest)
                else:
                    self.random_walk()
        else:
            if self.sense() == FOOD:
                self.take()
            else:
                path_to_food = self.find_target(FOOD)
                if path_to_food:
                    self.step_towards(path_to_food)
                else:
                    self.random_walk()
   

In [4]:
class Environment:
    def __init__(self, conf):    # lade eine Umgebung aus einer textuellen Beschreibung
        self.dimX = len(conf[0]) # Breite der Umgebung
        self.dimY = len(conf)    # Höhe der Umgebung
        self.floor = [[conf[j][i] if conf[j][i] != ANT else SPACE for i in range(self.dimX)] for j in range(self.dimY)]
        self.ant = Ant(self, 1, 1)
        for j in range(self.dimY):
            for i in range(self.dimX):
                if conf[j][i] == ANT:
                    self.ant.x = i
                    self.ant.y = j
        
    def setFloor(self, x, y, w): # setze ein Feld in der Umgebung
        self.floor[y][x] = w
        
    def getFloor(self, x, y): # gebe das Feld der Umgebung zurück
        return self.floor[y][x]
    
    def print(self): # drucke eine textuelle Beschreibung der Umgebung aus
        for y in range(self.dimY):
            for x in range(self.dimX):
                if self.ant.x == x and self.ant.y == y:
                    if self.ant.carryingFood():
                        print(ANT_WITH_FOOD, end='')
                    else:
                        print(ANT, end='')
                else:
                    print(self.getFloor(x, y), end='')
            print()
            
    def anyNestsLeft(self): # gibt es noch ein Nest, welches noch kein Futter erhalten hat?
        for y in range(self.dimY):
            for x in range(self.dimX):
                if self.floor[y][x] == NEST:
                    return True
        return False
    
    def tick(self): # führe einen Simulationsschritt durch
        self.ant.act()
        
    def simulate(self, verbose=False): # simuliere die Umgebung solange es noch Nester ohne Futter gibt
        steps = 0
        while self.anyNestsLeft():
            self.tick()
            steps += 1
            if verbose:
                self.print()
                time.sleep(0.05)
                clear_output(wait=True)
        return steps

In [5]:
scenario1 = [
"##########",
"#N       #",
"#        #",
"#        #",
"#        #",
"#    A   #",
"#        #",
"#        #",
"#       o#",
"##########"
]

scenario2 = [
"##########",
"#N       #",
"######   #",
"#A#      #",
"#   ######",
"#   #    #",
"#   # # ##",
"#   # #  #",
"#     # o#",
"##########"
]

env = Environment(scenario1)
env.simulate(True)
ant1 = Ant(env, 1, 1)
ant1.act()

umgebung = Environment(scenario2)
umgebung.simulate(True)
ant2 = Ant(env, 1, 1)
ant2.act()

##########
#A       #
######   #
# #      #
#   ######
#   #    #
#   # # ##
#   # #  #
#     #  #
##########


In [6]:
def simulate_multiple_times(environment_conf, runs=100):
    """Beschreibung: Implementierung einer Funktion, die den Mittelwert an zurueckgelegten Schritten berechnet
    Parameter: environment_conf: Eine Umgebung aus einer textuellen Beschreibung, runs: Anzahl an Wiederholungen mit dem Default-Wert 100
    Returns: Mittelwert an zurueckgelegten Schritten
    """
    total_steps = 0
    for _ in range(runs):
        env = Environment(environment_conf)
        steps = env.simulate(verbose=False)
        total_steps += steps
    return total_steps / runs

# Es wird die Simulation vom Szenario 1 100 wiederholt und anschließend der arithmetische Mittelwert der Anzahl an Schritten ermittelt
average_steps_scenario1 = simulate_multiple_times(scenario1)
print(f"Mittlere Anzahl von Schritten für scenario1: {average_steps_scenario1}")

# Es wird die Simulation vom Szenario 1 100 wiederholt und anschließend der arithmetische Mittelwert der Anzahl an Schritten ermittelt
average_steps_scenario2 = simulate_multiple_times(scenario2)
print(f"Mittlere Anzahl von Schritten für scenario2: {average_steps_scenario2}")

Mittlere Anzahl von Schritten für scenario1: 22.0
Mittlere Anzahl von Schritten für scenario2: 46.0


# Antworten

## **Aufgabe 3**

1. **Verwendeter Algorithmus**

    Der in dieser Aufgabe verwendete Suchalgorithmus ist die Breitensuche. Beim Maschinellen Lernen und in der Kuenstlichen Intelligenz werden 
    verschiedene Typen von Suchalgorithmen verwendet. Es gibt unter anderem die uniformierte Suche, zu der Tiefensuche und Breitensuche gehören, und 
    heuristische Suche, wie die Bergsteigersuche oder die Bestensuche. Die Breitensuche ist ein Algorithmus der uniformierten Suche, der 
    Schicht für Schicht jeden Knoten auf gleicher Tiefe im Suchbaum expandiert, bevor zu Knoten der nächsten Ebene uebergegangen wird. Durch diese 
    Eigenschaft ist die Breitensuche vollstaendig, wenn der Suchraum endlich ist, weil jeder Zustandsraum besucht wird.

2. **Funktionsweise der einzelnen Funktionen**

    Neben den bereits existierenden Membervariablen und Methoden werden die Membervariable *self.directions* sowie die Methoden *random_walk(self)*,  
    *find_target(self,target_type)*, *step_towards(self, path)* und *act(self)* implementiert.

    *self.directions* ist eine Liste mit vier Zustaenden, welche jeweils die Bewegung in eine der vier Himmelsrichtungen darstellt. 
    *random_walk(self)* basiert auf *self.directions* und verwendet die Methode *random.choice()*, um eine zufällige Bewegung der Ameise zu simulieren.

    Die Methode *find_target(self,target_type)* sucht nach dem nächstgelegenen Ziel einer bestimmten Art mit der Breitensuche. Wird 
    das Ziel gefunden und erreicht, endet das Programm. Falls kein Ziel gefunden wird, gibt die Funktion *None* zurück.

    Die Methode *step_towards(self, path)* fuehrt Bewegungen der Ameise entlang des gefundenen Pfades von *find_target* aus. Hierbei wird nicht 
    nach der kuerzesten Route gesucht, sondern der komplette Pfad abgeschritten, den die Breitensuche zur Verfuegung hat.

    Die Methode *act(self)* ist die tatsaechliche Implementierung der Aktionen der Ameise auf Basis der Breitensuche. Sie nutzt 
    die zuvor beschriebenen Methoden, um die Ameise in einem bestimmten Szenario eigenstaendig handeln zu lassen.

## **Aufgabe 4**

1. **Versuchsergebnisse**

    Beide Szenarien konnten relativ schnell und elegant geloest werden, welche Rechenzeiten von wenigen Sekunden zur Folge haben. 
    Das 2. Szenario hat einen etwa 3 mal so großen Rechenaufwand wie das 1. Szenario.
    
    Zudem wurde ueber dem Code-Block im Markdown-Format, mit dem Inhalt **"Antworten"**, eine Funktion aus dem ersten Teil der Modularbeit uebernommen, 
    die die durchschnittliche Anzahl an Schritten ermittelt. Die Funktion lautet **simulate_multiple_times(environment_conf, runs =100)**. Hierbei wird 
    ein beliebiges Szenario mit dem Default-Wert 100, also mittels 100 Wiederholugen, simuliert. Basierend auf Tests, braucht das 1. Szenario im Mittel 
    ca. 22 Schritte, waehrend die Anzahl an Schritten fuer das 2. Szenario mehr als doppelt so groß sind, etwa 46 Schritte.

2. **Evaluation**

    Die Gruende fuer die unterschiedlichen Ausgaenge fuer beide Szenarien liegen darin, dass das 1. Szenario kaum Hindernisse enthaelt, sodass 
    die Breitensuche ihre Aufgabe ideal und mit wenigen Schritten erfuellt. Das 2. Szenario ist wesentlich komplexer als das 1. Szenario, wodurch die 
    Bestensuche im 2. Szenario staerker beansprucht wird. Trotzdem wird das Ziel im 2. Szenario erreicht. Deshalb laesst sich feststellen, dass der 
    Agent sich ideal verhaelt. 